# Structures arborescentes, explorateur de fichiers

Nous avons déjà défini et travaillé aves les arbres binaires dans lesquels chaque noeud non-vide possède exécatement deux fils. On peut parfaitement concevoir des arbre plus généraux dans lesquels les noeuds ont un nombre quelconque de fils.

## 1 - Implémentation en python, fonctions de bases

Redéfinissons une classe `Noeud` de sorte de ne plus stocker des sous-arbres gauche et droit mais plutôt un tableau (éventuellement vide) de sous arbres.


In [13]:
class Noeud:
    """Arbres généraux"""
    def __init__(self, v, f = []):
        self.valeur = v
        self.fils = f
        
def taille(a):
    """renvoie le nombre de valeurs de l'arbre a, à compléter"""
    if a is None:
        return 0
    else:
        res = 1
        for f in a.fils:
            res+=taille(f)
        return res

def hauteur(a):
    """renvoie la hauteur de l'arbre"""
    if a is None:
        return 0
    elif a.fils == []:
        return 1
    else: 
        res = hauteur(a.fils[0])
        for i in range(1, len(a.fils)):
            h = hauteur(a.fils[i])
            if h>res:
                res=h
        return res+1
        
        
def parcours_prefixe(a):
    """affiche les valeurs de a dans l'ordre préfixe, à compléter"""
    if a is not None:
        print(a.valeur)
        for f in a.fils:
            parcours_prefixe(f)
       
def somme(a):
    """Renvoie la somme des éléments de a, à compléter"""
    if a is None:
        return 0
    else: 
        res = a.valeur
        for f in a.fils:
            res += somme(f)
        return res 

def recherche(a,v):
    """teste si la valeur v se trouve dans a, à compléter"""
    if a is None:
        return False
    elif a.valeur == v:
        return True 
    else:
        for f in a.fils: 
            if recherche(f, v):
                return True 
        return False
    
    
def minimum(a):
    """renvoie la plus petite valeur de a"""
    if a is None :
        raise IndexError
    else:
        return min([a.valeur]+[minimum(f) for f in a.fils])
        

#tests 
a = Noeud(1, [Noeud(2), Noeud(3, [Noeud(5, [Noeud(8), Noeud(9)]), Noeud(6, [])]), Noeud(4, [Noeud(7)])])
#tester vos fonctions 
print(taille(a))
print(hauteur(a))
parcours_prefixe(a)
print(somme(a))
print(recherche(a,9))
print(recherche(a,42))
print(minimum(a))


9
4
1
2
3
5
8
9
6
4
7
45
True
False
1


## 2 - Simulateur de terminal, système de fichiers

Sur les systèmes Linux, il est possible d'explorer et de gérer les fichiers et les dossiers directement dans un terminal. Les commandes principales sont les suivantes :

- ``ls`` (***l**i**s**t*) affiche les fichiers et dossiers du dossier courant ;
- ``cd <dossier>`` (*change directory*) permet de changer de dossier (``..`` pouvant faire référence au dossier parent) ;
- ``touch <nouveau_fichier>`` permet de créer un fichier vide ;
- ``mkdir <nouveau_dossier>`` (*make directory*) permet de créer un dossier vide. 




**A FAIRE :**
**tester ces commandes dans un terminal**



Le dossier courant est toujours affiché au début de la ligne :

``<utilisateur>@<machine> <chemin du dossier>``

On trouve souvent le symbole ``~`` pour désigner le dossier ``/home/<votre dossier personnel>``. 

Vous pouvez trouver plus de détails sur l'arborescence des fichiers Linux <a href = "https://www.malekal.com/les-repertoires-systemes-arborescence-linux/">ici</a>. 


### Activité

On se propose de recréer un Python un système d'arborescence de fichiers proposant des commande similaires


In [22]:
class Directory:
    """
    La classe dossier, similaire à Noeud mais comportant en plus une référence au dossier parent
    childs représente ce que contient le dossier, c'est un tableau de dossiers ou de fichiers
    """
    def __init__(self, name, parent):
        self.name = name
        self.parent = parent
        self.children = []
        
class File:
    """La classe fichier, définie par un nom, un dossier parent et un contenu"""
    def __init__(self, name, parent):
        self.name = name
        self.parent = parent
        self.content = ""
        
def is_dir(c):
    """Teste si c est un dossier"""
    return str(type(c)) == "<class '__main__.Directory'>"

def is_file(c):
    """Teste si c est un fichier"""
    return str(type(c)) == "<class '__main__.File'>"


ROOT = Directory("", None) #la racine de l'arborescence
DIR = ROOT #le dossier courant, modifié par la commande cd

def ls():
    """
    Affiche la liste des noms des dossiers et fichiers du dossier courant 
    """
    for c in DIR.children:
        print(c.name)

def cd(name):
    """
    change le dossier courant
    name doit être le nom d'un dossier contenu dans le dossier courant ou .. pour faire référence au parent
    """
    global DIR 
    if name == "..":
        DIR = DIR.parent
    else:
        for c in DIR.children: 
            if is_dir(c) and c.name == name: 
                DIR = c 
                break
    

def mkdir(name):
    """
    créer un nouveau dossier dans le dossier courant
    le nom name ne doit pas être déjà utilisé dans le dossier courant 
    """
    if name in [f.name for f in DIR.children]+[".."]:
        print("dossier ou fichier existant !")
    else:
        DIR.children.append(Directory(name, DIR))

def touch(name): 
    """
    créer un nouveau fichier dans le dossier courant
    le nom name ne doit pas être déjà utilisé dans le dossier courant 
    """
    if name in [f.name for f in DIR.children]+[".."]:
        print("dossier ou fichier existant !")
    else:
        DIR.children.append(File(name, DIR))

def edit(name, content):
    """
    modifie le contenu du fichier 
    """
    for c in DIR.children: 
        if c.name == name:
            c.content = content 
            break
        

def path(): 
    """renvoie le chemin de la racine vers le dossier courant sous la forme /dir1/dir2/..."""
    t = []
    r = DIR
    while r != ROOT:
        t.append(r.name)
        r = r.parent
    res = "/"
    for i in range(len(t)):
        res+=t[-1-i]
        if i!=len(t)-1:
            res+="/"
    return res

mkdir("dos1")
mkdir("dos2")
touch("config.txt")
cd("dos1")
mkdir("Documents")
cd("..")
ls()
cd("dos1")
cd("Documents")
print(path())


dos1
dos2
config.txt
/dos1/Documents


Une fois ces fonctions codées, créer une petite arborescence contenant à partir de la racine :
- un dossier ``programmes`` ;
- un dossier ``perso`` contenant des sous-dossiers ``Documents``, ``Images``, etc.